# Dataset Preparation

The dataset included as part of this repo is not ready for model training and evaluation. Because it contains the folded equivalent questions for each dataset split. You will need to unfold the equivalent questions to make them ready for the model development. Please run this notebook to prepare the dataset. You can then use the output dataset for model development.

In [1]:
import os
import pandas as pd
import re

In [2]:
def unfold_questions(df, column='folded_questions'):
    """
    Unfold all the folded equivalent questions in the given dataframe.
    Args:
        df(pd.DataFrame): Dataframe where it contains the data with folded questions.
    
    Returns:
        Dataframe with unfolded questions added.
    """
    
    def get_sentences_replacing_one_arg_with_options(sentence, match):
        start, end = match.start(), match.end()
        options = match.groups()[0].split("/")
        out_sentences = [sentence[:start] + option.strip() + sentence[end:] for option in options]
        return out_sentences

    
    SYN_ARG_P = re.compile("<SYN-ARG-(.*?)>")
    initial_columns = df.columns
    line_dfs = []

    # generate expanded dfs by row
    for _, row in df.iterrows():

        todo_sentences = [row[column]]
        out_sentences = []

        while len(todo_sentences)>0:
            next_sentence = todo_sentences.pop(0)
            
            args = list(re.finditer(SYN_ARG_P, next_sentence))
            if args:
                first_arg = args[0]
                option_sentences = get_sentences_replacing_one_arg_with_options(next_sentence, first_arg)
                todo_sentences.extend(option_sentences)
            else:
                out_sentences.append(next_sentence)

        # creat & append row dataframe
        row_df = pd.DataFrame({'unfolded_questions': out_sentences})
        for c in initial_columns: row_df[c] = row[c]
        line_dfs.append(row_df)

    # agg
    out_df = pd.concat(line_dfs, axis=0).reset_index(drop=True)
    return out_df

In [4]:
## Constants
DATA_DIR = '<path/to/input/data/directory>'  #Substitute with the actual input path
OUTPUT_DIR = '<path/to/output/data/directory>' #Substitute with the actual output path

TRAIN_PATH = os.path.join(DATA_DIR, 'train.csv')
VAL_PATH = os.path.join(DATA_DIR, 'validation.csv')
TEST_PATH = os.path.join(DATA_DIR, 'test.csv')

OUT_TRAIN_PATH = os.path.join(OUTPUT_DIR, 'train.csv')
OUT_VAL_PATH = os.path.join(OUTPUT_DIR, 'validation.csv')
OUT_TEST_PATH = os.path.join(OUTPUT_DIR, 'test.csv')

In [5]:
## Read Input Data

df_train = pd.read_csv(TRAIN_PATH)
df_val = pd.read_csv(VAL_PATH)
df_test = pd.read_csv(TEST_PATH)

#Total base questions (the same base questions are located in all dataset splits)
print('Total Base Questions:')
print(f'Train: {df_train.base_question.nunique()}, Val: {df_val.base_question.nunique()}, Test: {df_test.base_question.nunique()}\n')

total = df_train.shape[0] + df_val.shape[0] + df_test.shape[0]
print('Total Folded Equivalent Questions:')
print(f'Train: {df_train.shape[0]}, Val: {df_val.shape[0]}, Test: {df_test.shape[0]}, Total: {total}')
df_test.head()

Total Base Questions:
Train: 56, Val: 56, Test: 56

Total Folded Equivalent Questions:
Train: 533, Val: 125, Test: 131, Total: 789


,base_question,folded_questions,query
0,Count of patients grouped by race.,<SYN-ARG-number/count/counts> of <SYN-ARG-pati...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
1,Count of patients grouped by race.,<SYN-ARG-calculate/what's/what is/tell me> the...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
2,Counts of patients taking drug <ARG-DRUG><0> a...,<SYN-ARG-calculate/what's/what is/tell me> the...,SELECT COUNT( DISTINCT a.person_id ) FROM ((SE...
3,Counts of patients taking drug <ARG-DRUG><0> a...,How many <SYN-ARG-patients/people/persons/indi...,SELECT COUNT( DISTINCT a.person_id ) FROM ((SE...
4,Counts of patients taking drug <ARG-DRUG><0> a...,<SYN-ARG-calculate/what's/what is/tell me> the...,SELECT COUNT( DISTINCT a.person_id ) FROM ((SE...


In [6]:
## Unfold the questions for each split
print('Unfolding Questions...')
df_train = unfold_questions(df_train)
df_val = unfold_questions(df_val)
df_test = unfold_questions(df_test)
print('[SUCCESS]')

Unfolding Questions...
[SUCCESS]


In [8]:
#Total Dataset Sizes after Unfolding
total = df_train.shape[0] + df_val.shape[0] + df_test.shape[0]
print('Total Unfolded Equivalent Questions:')
print(f'Train: {df_train.shape[0]}, Val: {df_val.shape[0]}, Test: {df_test.shape[0]}, Total: {total}')
df_test.head()

Total Unfolded Equivalent Questions:
Train: 596961, Val: 145368, Test: 56931, Total: 799260


,unfolded_questions,base_question,folded_questions,query
0,number of patients in each race group.,Count of patients grouped by race.,<SYN-ARG-number/count/counts> of <SYN-ARG-pati...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
1,number of patients for each race group.,Count of patients grouped by race.,<SYN-ARG-number/count/counts> of <SYN-ARG-pati...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
2,number of people in each race group.,Count of patients grouped by race.,<SYN-ARG-number/count/counts> of <SYN-ARG-pati...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
3,number of people for each race group.,Count of patients grouped by race.,<SYN-ARG-number/count/counts> of <SYN-ARG-pati...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
4,number of persons in each race group.,Count of patients grouped by race.,<SYN-ARG-number/count/counts> of <SYN-ARG-pati...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."


In [9]:
## Save the Output Data

os.makedirs(OUTPUT_DIR, exist_ok=True)

df_train.to_csv(OUT_TRAIN_PATH, index=False)
df_val.to_csv(OUT_VAL_PATH, index=False)
df_test.to_csv(OUT_TEST_PATH, index=False)